# How to use text descriptors in text-specific metrics?

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite

from evidently.metric_preset import TextEvals

from evidently.metrics import TextDescriptorsDriftMetric
from evidently.metrics import TextDescriptorsDistribution
from evidently.metrics import TextDescriptorsCorrelationMetric
from evidently.metrics import ColumnDriftMetric, ColumnSummaryMetric

from evidently.descriptors import TextLength, IncludesWords, OOV, NonLetterCharacterPercentage, SentenceCount, WordCount, Sentiment, RegExp, SemanticSimilarity

In [ ]:
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

In [ ]:
#Dataset for Data Quality and Integrity
reviews_data = datasets.fetch_openml(name='Womens-E-Commerce-Clothing-Reviews', version=2, as_frame='auto')
reviews = reviews_data.frame

In [ ]:
reviews_ref = reviews[reviews.Rating > 3].sample(n=5000, replace=True, ignore_index=True, random_state=42)
reviews_cur = reviews[reviews.Rating < 3].sample(n=5000, replace=True, ignore_index=True, random_state=42)

In [ ]:
reviews.head()

In [ ]:
column_mapping = ColumnMapping(
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title']
)

# Text Metrics

In [ ]:
#NO descriptors

text_specific_metrics_report = Report(metrics=[
    TextDescriptorsDriftMetric(column_name="Review_Text"),
    TextDescriptorsDistribution(column_name="Review_Text"),
    TextDescriptorsCorrelationMetric(column_name="Review_Text"),
])

text_specific_metrics_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
text_specific_metrics_report

In [ ]:
#WITH descriptors

report = Report(metrics=[
    TextDescriptorsDriftMetric("Review_Text", descriptors={
        "Review Text Length" : TextLength(),
        "Reviews about Dress" : IncludesWords(words_list=['dress', 'gown']),
        "Review about Blouses" : IncludesWords(words_list=['blouse', 'shirt']),
        "Review Sentence Count" : SentenceCount(),
        "Review Word Count" : WordCount(),
        "Review Sentiment" : Sentiment(),
        "Review questions": RegExp(reg_exp=r'.*\?.*'),
    }),
    TextDescriptorsCorrelationMetric(column_name="Title", descriptors={
        "Title OOV" : OOV(),
        "Title Non Letter %" : NonLetterCharacterPercentage(),
        "Title Length" : TextLength(),
        "Title Sentence Count" : SentenceCount(),
        "Title Word Count" : WordCount(),
        "Title Sentiment" : Sentiment(),
        "Title questions": RegExp(reg_exp=r'.*\?.*'),
    })
])

report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
report


# Text Overview Preset

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="Review_Text")
    ]
)

text_evals_report.run(reference_data=reviews_ref[:100], current_data=reviews_cur[:100], column_mapping=column_mapping)
text_evals_report

In [ ]:
text_evals_report = Report(metrics=[
    TextEvals(column_name="Review_Text", descriptors=[
        SentenceCount(),
        WordCount(),
        Sentiment(),
        IncludesWords(words_list=['blouse', 'shirt']),
        ]
    )
])

text_evals_report.run(reference_data=reviews_ref[:100], current_data=reviews_cur[:100], column_mapping=column_mapping)
text_evals_report

In [ ]:
#NO descriptors
text_overview_report = Report(metrics=[
    TextEvals(column_name="Review_Text")
])

text_overview_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
text_overview_report

In [ ]:
#NO descriptors, several columns

text_overview_report = Report(metrics=[
    TextEvals(column_name="Review_Text"),
    TextEvals(column_name="Title"),
])

text_overview_report.run(reference_data=reviews_ref[:100], current_data=reviews_cur[:100], column_mapping=column_mapping)
text_overview_report

In [ ]:
#WITH descriptors
text_overview_report = Report(metrics=[
    TextEvals(column_name="Review_Text", descriptors=[
        OOV(),
        NonLetterCharacterPercentage(),
        TextLength(),
        IncludesWords(words_list=['dress', 'gown']),
        IncludesWords(words_list=['blouse', 'shirt']),
        SentenceCount(),
        WordCount(),
        Sentiment(),
        RegExp(reg_exp=r'.*\?.*'),
    ])
])

text_overview_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
text_overview_report

In [ ]:
text_overview_report = Report(metrics=[
    TextEvals(column_name="Review_Text", descriptors=[
        OOV(),
        NonLetterCharacterPercentage(),
        TextLength(),
        IncludesWords(words_list=['dress', 'gown']),
        IncludesWords(words_list=['blouse', 'shirt']),
        SentenceCount(),
        WordCount(),
        Sentiment(),
        RegExp(reg_exp=r'.*\?.*'),
    ]),
    TextEvals(column_name="Title", descriptors=[
        OOV(),
        NonLetterCharacterPercentage(),
        TextLength(),
        IncludesWords(words_list=['dress', 'gown']),
        IncludesWords(words_list=['blouse', 'shirt']),
        SentenceCount(),
        WordCount(),
        Sentiment(),
        RegExp(reg_exp=r'.*\?.*'),
    ])
])

text_overview_report.run(reference_data=reviews_ref[:100], current_data=reviews_cur[:100], column_mapping=column_mapping)
text_overview_report

# Column Drift Metric for Text Descriptors

In [ ]:
drift_report = Report(metrics=[
    ColumnDriftMetric(column_name=TextLength(display_name="TextLength").for_column("Review_Text")),
    ColumnDriftMetric(column_name=IncludesWords(words_list=['blouse', 'shirt'], display_name="Review about Blouses").for_column("Review_Text")),

])

drift_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
drift_report

In [ ]:
summary_report = Report(metrics=[
    ColumnSummaryMetric(column_name=SemanticSimilarity(with_column="Title").on("Review_Text"))
])

summary_report.run(reference_data=reviews_ref[:10], current_data=reviews_cur[:10], column_mapping=column_mapping)
summary_report